In [1]:
#%load_ext autoreload
#%autoreload 2
%matplotlib inline
%cd '../'

/Users/laurentperrinet/quantic/science/HomeHots/HOTS_clone_laurent


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
# 
# https://en.wikipedia.org/wiki/ISO_8601
import datetime
timestr = datetime.datetime.now().date().isoformat()
timestr = '20201021'
tau = 9e-4

from HOTS.Event import conv2eve
from HOTS.Tools import SaveObject,LoadObject
DataPath = 'Data/testsetnmnist.p'
NbClusteringData = 2500
NbTrainingData = 5000
NbTestingData = 1500

fname_event_nmnist = 'Records/EXP_03_NMNIST/'+timestr+'_hots_event_nmnist.p'
try:
    dataset = LoadObject(fname_event_nmnist)
except:
    from HOTS.Event import LoadNMNIST
    dataset = LoadNMNIST(NbTrainingData, NbTestingData, NbClusteringData, 
                         Path=DataPath, OneOfEach=False, 
                         OutOnePolarity=False, ListPolarities=None, verbose=0)
    SaveObject(dataset, fname_event_nmnist)

events_train, events_test, event_cl, labels_train, labels_test = dataset


In [3]:
def get_events(timestr, 
               tau=tau, # -> tau=1ms, si on prend 10 ms on est à 1s pour la dernière couche et les vidéos font 0.3s en moyenne
               hom=True, verb=False):
    R = 2    
    filthr = 2
    nbkNN = 3
    algo = 'lagorce'
    decay = 'exponential'
    hom = True
    krnlinit = 'rdn'
    nb_cluster = [4, 8, 16]

    if hom==True:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'_homeo.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'_event0_o_homeo.pkl'
    else:
        fname_model = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'ms_'+algo+'.pkl'
        fname_event0_o = 'Records/EXP_03_NMNIST/'+timestr+'_hots_'+str(tau*1000)+'_event0_o.pkl'

    if not os.path.isfile(fname_event0_o):            
        print('creating the events in file', fname_event0_o)
        from HOTS.ToolsMonitor import GenerateActivationMap, DisplayActivationMap
        from HOTS.Event import Event, SimpleAlphabet, LoadNMNIST
        from HOTS.STS import STS
        from HOTS.Layer import ClusteringLayer
        from HOTS.ToolsMonitor import (
            DisplayImage,
            DisplaySurface3D,
            DisplaySurface2D,
            DisplayConvergence,
        )
        from HOTS.Tools import SaveObject, LoadObject
        from HOTS.Network import Network
        from HOTS.Event import conv2eve

        event_tr, event_te, event_cl, label_tr, label_te = LoadNMNIST(
        NbTrainingData, NbTestingData, NbClusteringData, OneOfEach=False, Path=DataPath, OutOnePolarity=False, ListPolarities=None, verbose=0)

        L1 = ClusteringLayer(tau=tau,R=R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
        L2 = ClusteringLayer(tau=10 * tau,R=2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
        L3 = ClusteringLayer(tau=10 * 10 * tau,R=2 * 2 * R,verbose=0,ThrFilter=filthr,LearningAlgo=algo,kernel=decay,homeo=hom,init=krnlinit)
        Net = Network([L1, L2, L3])    

        if not os.path.isfile(fname_model):
            ClusterLayer, event_output = Net.TrainCluster(
                    event=event_cl, NbClusterList=nb_cluster, to_record=True, NbCycle=1
                )
            SaveObject(ClusterLayer, fname_model)
        else: 
            print('loading model from file', fname_model)

            ClusterLayer, Classif0 = LoadObject(fname_model)

        print('run the events through the network')
        homrun = False
        events_train_o = Net.RunNetwork(event_tr, NbClusterList=ClusterLayer, homrun=homrun)
        events_test_o = Net.RunNetwork(event_te, NbClusterList=ClusterLayer, homrun=homrun)
        SaveObject([events_train, events_test], fname_event0_o)
    else: 
        if verb: print('loading the events from file', fname_event0_o)
        events_train_o, events_test_o = LoadObject(fname_event0_o)
        
    return events_train_o, events_test_o


In [ ]:
#X_train, y_train = gather_data(events_train, labels_train)
events_train_o, events_test_o = get_events(timestr, tau=tau, hom=True, verb=True, )

creating the events in file Records/EXP_03_NMNIST/20201021_hots_0.9_event0_o_homeo.pkl
[]
loading model from file Records/EXP_03_NMNIST/20201021_hots_0.9ms_lagorce_homeo.pkl
run the events through the network


### Building matrix for logistic regression

In [ ]:
def gather_data(events, labels, 
                tau_cla = .300, # characteristic time of a digit 
                sample_events = 200,    sample_space = 1,
                verbose=False):
    c_int = lambda n, d : ((n - 1) // d) + 1

    n_events = events.time.shape[0]

    data = np.zeros((c_int(events.ImageSize[0], sample_space), 
                     c_int(events.ImageSize[1], sample_space), 
                     len(events.ListPolarities))) #tmp data

    X = np.zeros((c_int(n_events, sample_events), len(data.ravel())))
    y = np.zeros((c_int(n_events, sample_events), ))

    t_absolute = 0
    t_old = 0
    i_labels = 0
    for i in range(1, n_events):
        if i == events.ChangeIdx[i_labels]:
            i_labels += 1
            t_absolute += events.time[i]
        t_new = t_absolute + events.time[i]
        
        data *= np.exp(-(t_new-t_old)/tau_cla)
        
        t_old = t_new

        data[events.address[i, 0]//sample_space, 
             events.address[i, 1]//sample_space, 
             events.polarity[i]] = 1.

        if i % sample_events == 0:
            if verbose: print(f'{i=} {n_events=} {n_events//sample_events=}')
            X[i//sample_events, :] = data.ravel()
            y[i//sample_events] = labels[i_labels]

    print('Number of events: ' + str(X.shape[0])+' - Number of features: ' + str(X.shape[1]))

    return X, y


In [ ]:
from sklearn.linear_model import LogisticRegression as LR
# from sklearn.linear_model import LogisticRegressionCV as LR
from sklearn.model_selection import train_test_split

opts_LR = dict(random_state=0, max_iter=5000, n_jobs=32, class_weight='balanced')
#opts_LR['Cs'] = 5
#opts_LR['Cs'] = 32

# directly from the input
# X_train, y_train = gather_data(events_train, labels_train)
# lr = LogisticRegressionCV(**opts_LR).fit(X_train, y_train)
# X_test, y_test = gather_data(events_test, labels_test)
# print(f'Classification score for dataset is {lr.score(X_test, y_test):.3f}')

In [ ]:
# events_train, event_test, event_cl, labels_train, labels_test = dataset

X, y = gather_data(events_train, labels_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')



In [ ]:
# events_train, event_test, event_cl, labels_train, labels_test = dataset

X_train, y_train = gather_data(events_train, labels_train)
lr = LR(**opts_LR).fit(X_train, y_train)
print(f'Classification score for raw input is {lr.score(X_train, y_train):.3f}')
X_test, y_test = gather_data(events_test, labels_test)
print(f'Classification score for raw input is {lr.score(X_test, y_test):.3f}')

